train face model

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
f=pd.read_csv('/content/gdrive/My Drive/faceData/mainFaceData.csv')
ft=pd.read_csv('/content/gdrive/My Drive/faceData/mainTestFaceData.csv')
x=f.loc[:,:'783']
y=f.loc[:,'labels']
x=np.array(x)
y=np.array(y)
y=tf.one_hot(y,2,axis=-1)
with tf.Session() as sess:
    y=sess.run(y)
xt=ft.loc[:,:'783']
yt=ft.loc[:,'labels']
xt=np.array(xt)
yt=np.array(yt)
yt=tf.one_hot(yt,2,axis=-1)
with tf.Session() as sess:
    yt=sess.run(yt)
num0=np.arange(len(x))
num1=np.arange(len(xt))
np.random.shuffle(num0)
np.random.shuffle(num1)
new_x=[]
new_y=[]
for i in num0:
    new_x.append(x[i])
    new_y.append(y[i])
x=np.array(new_x)
y=np.array(new_y)
new_xt=[]
new_yt=[]
for i in num1:
    new_xt.append(xt[i])
    new_yt.append(yt[i])
xt=np.array(new_xt)
yt=np.array(new_yt)
label_dict={0:'lokesh',1:'notLokesh'}
plt.subplot(121)
x_img=np.reshape(x[0],(28,28))
plt.imshow(x_img,cmap='gray')
plt.title(label_dict[np.argmax(y[0])])
plt.subplot(122)
x_img=np.reshape(x[8000],(28,28))
plt.imshow(x_img,cmap='gray')
plt.title(label_dict[np.argmax(y[8000])])
plt.show()
training_iter=1
x_train=np.reshape(x,(-1,28,28,1))
y_train=y
x_test=np.reshape(xt,(-1,28,28,1))
y_test=yt
X=tf.placeholder(tf.float32,shape=[None,28,28,1])
Y=tf.placeholder(tf.float32,shape=[None,2])
def conv_layer(x,w,b):
    conv=tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')
    bias=tf.nn.bias_add(conv,b)
    return tf.nn.relu(bias)
def maxpool_layer(x,k=2):
    maxpool=tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    return maxpool
w0=tf.Variable(tf.random_normal(shape=[5,5,1,32]),tf.float32,name='w0')
w1=tf.Variable(tf.random_normal(shape=[5,5,32,64]),tf.float32,name='w1')
w2=tf.Variable(tf.random_normal(shape=[5,5,64,128]),tf.float32,name='w2')
wd1=tf.Variable(tf.random_normal(shape=[4*4*128,128]),tf.float32,name='wd1')
wout=tf.Variable(tf.random_normal(shape=[128,2]),tf.float32,name='wout')
b0=tf.Variable(tf.random_normal(shape=[32]),tf.float32,name='b0')
b1=tf.Variable(tf.random_normal(shape=[64]),tf.float32,name='b1')
b2=tf.Variable(tf.random_normal(shape=[128]),tf.float32,name='b2')
bd1=tf.Variable(tf.random_normal(shape=[128]),tf.float32,name='bd1')
bout=tf.Variable(tf.random_normal(shape=[2]),tf.float32,name='bout')
conv1=conv_layer(X,w0,b0)
conv1=maxpool_layer(conv1)
conv2=conv_layer(conv1,w1,b1)
conv2=maxpool_layer(conv2)
conv3=conv_layer(conv2,w2,b2)
conv3=maxpool_layer(conv3)
fc1=tf.reshape(conv3,[-1,wd1.get_shape().as_list()[0]])
fc1=tf.add(tf.matmul(fc1,wd1),bd1)
fc1=tf.nn.relu(fc1)
out=tf.add(tf.matmul(fc1,wout),bout)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,labels=Y),name='cost')
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
correct_pred=tf.equal(tf.argmax(out,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')
init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    train_accuracy=[]
    test_accuracy=[]
    train_loss=[]
    test_loss=[]
    summary_writer=tf.summary.FileWriter('./output',sess.graph)
    for epoch in range(training_iter):
        for batch in range(0,len(x_train),32):
            batch_x=x_train[batch:batch+32,:]
            batch_y=y_train[batch:batch+32,:]
            opt=sess.run(optimizer,feed_dict={X:batch_x,Y:batch_y})
            loss,acc=sess.run([cost,accuracy],feed_dict={X:batch_x,Y:batch_y})
        print('iter:',epoch,'loss:',loss,'training accuracy:',acc)
        test_acc,valid_loss=sess.run([accuracy,cost],feed_dict={X:x_test,Y:y_test})
        train_loss.append(loss)
        train_accuracy.append(acc)
        test_loss.append(valid_loss)
        test_accuracy.append(test_acc)
        print('testing accuracy:',test_acc)
        #saver.save(sess,'/content/gdrive/My Drive/faceModel',global_step=1)
    summary_writer.close()
    saver.save(sess,'/content/gdrive/My Drive/faceModel',global_step=1)
plt.plot(range(training_iter),train_loss,'b',label='training loss')
plt.plot(range(training_iter),test_loss,'r',label='test loss')
plt.title('training and test loss')
plt.xlabel('epochs',fontsize=16)
plt.ylabel('loss',fontsize=16)
plt.legend()
plt.figure()
plt.plot(range(training_iter),train_accuracy,'b',label='training accuracy')
plt.plot(range(training_iter),test_accuracy,'r',label='test accuracy')
plt.title('training and test accuracy')
plt.xlabel('epochs',fontsize=16)
plt.ylabel('accuracy',fontsize=16)
plt.legend()
plt.show()
print(X,Y,accuracy,cost)

test face model

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
ft=pd.read_csv('/content/gdrive/My Drive/mainTestFaceData.csv')
xt=ft.loc[:,:'783']
yt=ft.loc[:,'labels']
xt=np.array(xt)
yt=np.array(yt)
yt=tf.one_hot(yt,2,axis=-1)
with tf.Session() as sess:
    yt=sess.run(yt)
x_test=np.reshape(xt,(-1,28,28,1))
y_test=yt
#for op in tf.get_default_graph().get_operations():
#  print(op.name)
#latest_ckp = tf.train.latest_checkpoint('/content/gdrive/My Drive/')
#print_tensors_in_checkpoint_file(latest_ckp, all_tensors=False, tensor_name='')
with tf.Session() as sess:
  saver=tf.train.import_meta_graph('/content/gdrive/My Drive/faceModel-1.meta')
  saver.restore(sess,tf.train.latest_checkpoint('/content/gdrive/My Drive/'))
  graph=tf.get_default_graph()
  X=graph.get_tensor_by_name('Placeholder_5:0')
  Y=graph.get_tensor_by_name('Placeholder_6:0')
  accuracy=graph.get_tensor_by_name('accuracy:0')
  cost=graph.get_tensor_by_name('cost:0')
  test_acc,valid_loss=sess.run([accuracy,cost],feed_dict={X:x_test,Y:y_test})
  print(test_acc,valid_loss)